In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import userinput
from IPython.display import clear_output


In [ ]:
df_temp_time = pd.read_csv('csv_and_xlsx/LT_Analysis_I_temp_time.csv')

time_index_H = pd.date_range(start=str(userinput.year_input) +"-01-01 00:00", end=str(userinput.year_input) +"-12-31  23:00", freq="H")
temperature_heat = pd.Series(df_temp_time['temperature_heat'].values, index=time_index_H)
heat_demand = pd.Series(df_temp_time['heat_demand'].values, index=time_index_H)
COP_hp = pd.Series(df_temp_time['COP_hp'].values, index=time_index_H)
electricity_price = pd.Series(df_temp_time['electricity_price'].values, index=time_index_H)

In [ ]:
df_Analysis_I = pd.read_csv("csv_and_xlsx/LT_ATES_sensitivity_results_I.csv")
if userinput.Depth_drilling_possible:
    pass
else:
    df_Analysis_I = df_Analysis_I[df_Analysis_I['depth_ATES'] == df_Analysis_I['depth_ATES'].min()]


first_row = df_Analysis_I.iloc[[0]]
max_phi_row = df_Analysis_I.loc[[df_Analysis_I['phi_standing'].idxmax()]]
min_phi_row = df_Analysis_I.loc[[df_Analysis_I['phi_standing'].idxmin()]]

df_Analysis_II = pd.concat([first_row, max_phi_row, min_phi_row]).drop_duplicates().reset_index(drop=True)

In [ ]:
import os
from ATES_optimization import optimization

# Define the ranges for each variable
phi_s_range = df_Analysis_II['phi_standing'].values # Assuming phi_standing is the variable of interest
phi_c_range = df_Analysis_II['phi_charge_ATES'].values
elpr_range = np.array(userinput.elpr_range)
drc_range = np.array(userinput.drc_range)
CapTT_range = np.array(userinput.CapTT_range) 

if userinput.Depth_drilling_possible:
    os.makedirs("LT_optimization_2_results_depth", exist_ok=True)
    # Delete all files in the directory
    for f in os.listdir("LT_optimization_2_results_depth"):
        file_path = os.path.join("LT_optimization_2_results_depth", f)
        if os.path.isfile(file_path):
            os.remove(file_path)
else:
    os.makedirs("LT_optimization_2_results", exist_ok=True)
    # Delete all files in the directory
    for f in os.listdir("LT_optimization_2_results"):
        file_path = os.path.join("LT_optimization_2_results", f)
        if os.path.isfile(file_path):
            os.remove(file_path)

            
for phi_s in phi_s_range:
    for elpr in elpr_range:
        for drc in drc_range:
            for CapTT in CapTT_range:
                # Run the optimization with the current parameters
                df_aux = df_Analysis_II[df_Analysis_II['phi_standing'] == phi_s]
                m, obj_sol = optimization(df_aux.depth_ATES.iloc[0], df_aux.ATES_max_power.iloc[0], time_index_H, temperature_heat,df_aux.phi_standing.iloc[0], df_aux.phi_pipes.iloc[0], 
                                    df_aux.phi_charge_ATES.iloc[0], heat_demand, df_aux.P_pump.iloc[0], COP_hp, elpr * electricity_price, drilling_costs=drc, Capex_TTES=CapTT, interest=userinput.interest
                                )
                # Collect all results from m.solution and m.dual in a DataFrame
                # Combine m.solution and m.dual into a single DataFrame with matching datetime index
                m_solution = pd.DataFrame({k: v.values for k, v in m.solution.data_vars.items()}, index=time_index_H)
                m_dual = pd.DataFrame({k: v.values for k, v in m.dual.data_vars.items()}, index=time_index_H)
                system_info = pd.DataFrame({
                    'phi_standing': [phi_s],
                    'electricity_price': [elpr],
                    'drilling_costs': [drc],
                    'Capex_TTES': [CapTT],
                }, index = time_index_H)  
                
                for k, v in obj_sol.items():
                    system_info[k] = v

                # Concatenate both DataFrames horizontally
                df_result = pd.concat([m_solution, m_dual, system_info], axis=1)
                df_result.reset_index(inplace=True)
                df_result.rename(columns={'index': 'datetime'}, inplace=True)
                # Create filename with short variable names and values
                if userinput.Depth_drilling_possible:
                    filename = f"LT_optimization_2_results_depth/phi_s_{round(phi_s,6)}_elp_{round(elpr,2)}_drc_{round(drc,2)}_CapTT_{round(CapTT,2)}.csv"
                else:
                    filename = f"LT_optimization_2_results/phi_s_{round(phi_s,6)}_elp_{round(elpr,2)}_drc_{round(drc,2)}_CapTT_{round(CapTT,2)}.csv"
                df_result.to_csv(filename, index=False)

clear_output()
